# Setup

In [2]:
import requests
from lxml import html
import pandas as pd
from pprint import pprint
from tqdm import tqdm_notebook
import time
from datetime import datetime, timedelta

In [9]:
# Load confidential credential data
import aml_private

apikey = aml_private.api_key
url_template = 'https://api.companieshouse.gov.uk{}'
request_count = {'count':0, 'reset_time':datetime.now()}

# Define functions

In [37]:
# Request from api whilst counting total requests in 5 minute period in line with CH rate limit

def request_api(url, apikey, counter):
    # Remaining time of 5 min period since last reset else 0 if more than 5 mins since last reset 
    time_remaining = max(5*60-(datetime.now()-counter['reset_time']).total_seconds(), 0)
    
    if time_remaining==0:
        # Reset counter and make request
        counter['count']=1
        counter['reset_time']=datetime.now()
    
    elif counter['count']<500:
        # Increment counter and make request
        counter['count']+=1
        output = requests.get(url, auth=(apikey, '')).json()
    
    else:
        # Wait until end of period then reset counter and make request
        time.sleep(time_remaining)
        counter['count']=1
        counter['reset_time']=datetime.now()
        output = requests.get(url, auth=(apikey, '')).json()
        print(counter)
    
    # Update counter and output request
    return output, counter

In [4]:
# Scrape information from company page

def fetch_comp_info(link, url_template, apikey, counter):
    # Formulate url and make request
    url = url_template.format(link)
    r, counter = request_api(url, apikey, counter) 
    
    # Collect basic company information
    name = r['company_name']
    comp_number = r['company_number']
    kind = r['type']
    creation_date = r['date_of_creation']
    
    
    # Check for various address elements
    try:
        address = r['registered_office_address']['address_line_1']
    except:
        address = ''
    try:
        address += ' ' + r['registered_office_address']['address_line_2']
    except:
        pass
    try:
        address += ' ' + r['registered_office_address']['locality']
    except:
        pass
    try:
        address += ' ' + r['registered_office_address']['country']
    except:
        pass
    
    # Check if company has been dissolved
    try:
        dissolved_date = r['date_of_cessation']
    except:
        dissolved_date = ''
    
    return name, comp_number, address, kind, creation_date, dissolved_date, counter

In [5]:
# Scrape officer information from company's officers page

def fetch_officers(comp_number, url_template, apikey, counter):
    # Formulate url from company number and make request
    url = url_template.format('/company/{}/officers'.format(comp_number))
    r, counter = request_api(url, apikey, counter)
    
    # Instantiate lists for each information category
    officer_names = []
    officer_roles = []
    officer_types = []
    officer_statuses = []
    officer_countries = []
    officer_addresses = []
    officer_comp_numbers = []

    # Check for active officers
    try:
        num_active = r['active_count']
    except KeyError:
        num_active = 0
    
    # Check for inactive officers
    try:
        num_inactive = r['inactive_count']
    except KeyError:
        num_inactive = 0
    
    # Check for resigned officers
    try:
        num_resigned = r['resigned_count']
    except KeyError:
        num_resigned = 0
    
    # For companies with at least one officer
    if r != {}:
        # Iterate through each officer
        for i in r['items']:
            
            # Look for name
            try:
                officer_name = i['name']
            except:
                officer_name = ''
                
            # Look for role
            try:
                officer_role = i['officer_role']
            except:
                officer_role = ''

            # Set status as resigned if resignation date present
            try:
                i['resigned_on']
                officer_status = 'resigned'
            except:
                officer_status = 'active'
                
            # Check for country
            try:
                officer_country = i['address']['country']
            except:
                officer_country = ''

            # Look for date of birth to assign type as person
            try:
                i['date_of_birth']
                officer_type = 'person'
            except:
                officer_type = 'company'
             
            # Look for address
            try:
                officer_address = i['address']['address_line_1']
            except:
                officer_address = ''
            try:
                officer_address += ' ' + i['address']['address_line_2']
            except:
                pass
            try:
                officer_address += ' ' + i['address']['locality']
            except:
                pass
            try:
                officer_address += ' ' + i['address']['country']
            except:
                pass

            # Add details for current officer to list
            officer_names.append(officer_name)
            officer_roles.append(officer_role)
            officer_types.append(officer_type)
            officer_statuses.append(officer_status)
            officer_countries.append(officer_country)
            officer_addresses.append(officer_address)
            officer_comp_numbers.append(comp_number)
    
    return num_active, num_inactive, num_resigned, officer_names, officer_roles, officer_types,\
    officer_statuses, officer_countries, officer_addresses, officer_comp_numbers, counter

In [6]:
# Scrape PSC information from company's PSC/RLE page

def fetch_psc(comp_number, url_template, apikey, counter):
    # Formulate url from company number and make request
    url = url_template.format('/company/{}/persons-with-significant-control'.format(comp_number))
    r, counter = request_api(url, apikey, counter)
    
    # Instantiate lists of PSC details
    psc_types = []
    psc_statuses = []
    psc_countries = []
    psc_comp_numbers = []
    
    try:
        # Check for number of active and number of ceased PSCs
        num_active = r['active_count']
        num_ceased = r['ceased_count']
        
    except:
        num_active = 0
        num_ceased = 0
        
    # For companies with at least one PSC
    if r!={}:
        # Iterate through PSCs
        for i in r['items']:
            
            # Check for ceased date to define status
            try:
                i['ceased_on']
                psc_status = 'ceased'
            except:
                psc_status = 'active'
            
            # Check for date of birth to assign type as person
            try:
                i['date_of_birth']
                psc_type = 'person'
            except:
                try:
                    psc_type = i['identification']['legal_form']
                except:
                    psc_type = ''
            
            # Check for country
            try:
                psc_country = i['address']['country']
            except:
                psc_country = ''

            # Update list with details of current PSC
            psc_types.append(psc_type)
            psc_statuses.append(psc_status)
            psc_countries.append(psc_country)
            psc_comp_numbers.append(comp_number)

        
    return num_active, num_ceased, psc_types, psc_statuses, psc_countries, psc_comp_numbers, counter

In [49]:
# Scrape filing information from company's filing history page

def fetch_filings(comp_number, url_template, apikey, counter):
    # Formulate url and make request
    url = url_template.format('/company/{}/filing-history'.format(comp_number))
    r, counter = request_api(url, apikey, counter)
    
    # Instatiate lists for filings information
    comp_numbers = []
    filing_dates = []
    filing_descriptions = []
    name_change = []
    
    # Define starting point for iterating through multiple pages
    start_index = 0
    
    # Check if current page is last page
    while int(r['start_index']) + int(r['items_per_page']) < r['total_count']:
        # Check items per page, if final page then count of actual items on page 
        r_items = min(r['items_per_page'], r['total_count']-r['start_index'])
        
        # Iterate through each result
        for i in range(r_items):
            # Add company number as identifier
            comp_numbers.append(comp_number)
            
            # Collect date and description of each filing
            filing_dates.append(r['items'][i]['date'])
            filing_descriptions.append(r['items'][i]['description'])
            
            # Check if filing is company name change
            try:
                # If a resolution check if type name change
                if r['items'][i]['resolutions'][0]['category'] == 'change-of-name':
                    name_change.append(1)
                else:
                    name_change.append(0)
            
            # If not a resolution, mark as not a change of name
            except:
                name_change.append(0)
        
        # Increment to next page of results, reformat url and make request
        start_index += 25
        suffix = '/company/{}/filing-history?start_index={}'.format(comp_number, start_index)
        url = url_template.format(suffix)
        r, counter = request_api(url, apikey, counter)
        
    return comp_numbers, filing_dates, filing_descriptions, name_change, counter

In [7]:
# Perform search from a list of company names and return 4 relational dataframes  

def company_search(companies, url_template, apikey, counter):
    
    # Define company result lists
    names = []
    comp_numbers = []
    addresses = []
    kinds = []
    creation_dates = []
    dissolved_dates = []
    active_officers = []
    inactive_officers = []
    resigned_officers = []
    active_pscs = []
    ceased_pscs = []

    # Define officer result lists
    officer_names = []
    officer_roles = []
    officer_types = []
    officer_statuses = []
    officer_countries = []
    officer_addresses = []
    officer_comp_numbers = []
    
    # Define PSC result lists
    psc_types = []
    psc_statuses = []
    psc_countries = []
    psc_comp_numbers = []
    
    # Define filing history result lists
    filing_dates = []
    filing_descriptions = []
    filing_comp_numbers = []
    filing_name_changes = []

    # Iterate through company names
    for company in tqdm_notebook(companies):
        # Formulate search url and make request
        suffix = '/search/companies?q="{}"'.format(company.replace(' ', '+'))
        url = url_template.format(suffix)
        r, counter = request_api(url, apikey, counter)
        
        # Count number of items on page
        r_items = min(r['items_per_page'], r['total_results'])

        # Iterate through each company in search results 
        for i in range(r_items):
        
            # Check company name is a match
            title = r['items'][i]['title'].strip().replace('  ', ' ')
            if company.lower() in title.lower():
                link = r['items'][i]['links']['self']    

                # Fetch company information
                name, comp_number, address, kind, creation_date, dissolved_date, counter =\
                fetch_comp_info(link, url_template, apikey, counter)

                # Update company result lists for current company
                names.append(name.lower())
                comp_numbers.append(comp_number)
                addresses.append(address)
                kinds.append(kind)
                creation_dates.append(creation_date)  
                dissolved_dates.append(dissolved_date)

                # Fetch officer information
                num_active, num_inactive, num_resigned,\
                officer_name, officer_role, officer_type, officer_status, officer_country,\
                officer_address,officer_comp_number, counter =\
                fetch_officers(comp_number, url_template, apikey, counter)

                # Update officer result lists for current company
                active_officers.append(num_active)
                inactive_officers.append(num_inactive)
                resigned_officers.append(num_resigned)
                officer_names.append(officer_name)
                officer_roles.append(officer_role)
                officer_types.append(officer_type)
                officer_statuses.append(officer_status)
                officer_countries.append(officer_country)
                officer_addresses.append(officer_address)
                officer_comp_numbers.append(officer_comp_number)

                # Fetch PSC information
                psc_active, psc_ceased, psc_type, psc_status, psc_country, psc_comp_number, counter =\
                fetch_psc(comp_number, url_template, apikey, counter)

                # Update PSC result lists for current company
                active_pscs.append(psc_active)
                ceased_pscs.append(psc_ceased)
                psc_types.append(psc_type)
                psc_statuses.append(psc_status)
                psc_countries.append(psc_country)
                psc_comp_numbers.append(psc_comp_number)

                # Fetch filing history
                filing_comp_number, filing_date, filing_description, filing_name_change counter =\
                fetch_filings(comp_number, url_template, apikey, counter)

                # Update filing result lists for current company
                filing_comp_numbers.append(filing_comp_number)
                filing_dates.append(filing_date)
                filing_descriptions.append(filing_description)
                filing_name_changes.append(filing_name_change)

            # Stop searching once results no longer match company name
            else:
                break
                    

    # Flatten lists
    officer_names = [i for sublist in officer_names for i in sublist]
    officer_roles = [i for sublist in officer_roles for i in sublist]
    officer_types = [i for sublist in officer_types for i in sublist]
    officer_statuses = [i for sublist in officer_statuses for i in sublist]
    officer_countries = [i for sublist in officer_countries for i in sublist]
    officer_addresses = [i for sublist in officer_addresses for i in sublist]
    officer_comp_numbers = [i for sublist in officer_comp_numbers for i in sublist]
    psc_types = [i for sublist in psc_types for i in sublist]
    psc_statuses = [i for sublist in psc_statuses for i in sublist]
    psc_countries = [i for sublist in psc_countries for i in sublist]
    psc_comp_numbers = [i for sublist in psc_comp_numbers for i in sublist]
    filing_comp_numbers = [i for sublist in filing_comp_numbers for i in sublist]
    filing_dates = [i for sublist in filing_dates for i in sublist]
    filing_descriptions = [i for sublist in filing_descriptions for i in sublist]


    # Create company results dataframe
    company_results = pd.DataFrame({'name':names,
                    'address':addresses,
                    'kind':kinds,
                    'company_number':comp_numbers,
                    'creation_date':creation_dates,
                    'dissolved_date':dissolved_dates,
                    'active_officers':active_officers,
                    'inactive_officers':inactive_officers,
                    'resigned_officers': resigned_officers,
                    'active_psc': active_pscs,
                    'ceased_psc': ceased_pscs})

    # Create officer results dataframe
    officer_results = pd.DataFrame({'company_number': officer_comp_numbers,
                                    'name': officer_names,
                                    'type': officer_types,
                                    'role': officer_roles,
                                    'status': officer_statuses,
                                    'country': officer_countries,
                                    'address': officer_addresses})
    
    # Create PSC results dataframe
    psc_results = pd.DataFrame({'company_number': psc_comp_numbers,
                            'type': psc_types,
                            'status': psc_statuses,
                            'country': psc_countries})
    
    # Create filing results dataframe
    filing_results = pd.DataFrame({'company_number': filing_comp_numbers,
                                  'filing_date': filing_dates,
                                  'description': filing_descriptions,
                                  'name_change': filing_name_changes})


    return company_results, officer_results, psc_results, filing_results, counter

In [8]:
# Perform search from a list of company numbers and return 4 relational dataframes

def company_number_search(company_nums, url_template, apikey, counter):
    
    # Define company result lists
    names = []
    comp_numbers = []
    addresses = []
    kinds = []
    creation_dates = []
    dissolved_dates = []
    active_officers = []
    inactive_officers = []
    resigned_officers = []
    active_pscs = []
    ceased_pscs = []

    # Define officer result lists
    officer_names = []
    officer_roles = []
    officer_types = []
    officer_statuses = []
    officer_countries = []
    officer_addresses = []
    officer_comp_numbers = []
    
    # Define psc result lists
    psc_types = []
    psc_statuses = []
    psc_countries = []
    psc_comp_numbers = []
    
    # Define filing history result lists
    filing_dates = []
    filing_descriptions = []
    filing_comp_numbers = []
    filing_name_changes = []

    # Iterate through company numbers
    for company_num in tqdm_notebook(company_nums):
        
        # Formulate search url
        link = '/company/{}'.format(company_num)

        # Fetch company information
        name, comp_number, address, kind, creation_date, dissolved_date, counter =\
        fetch_comp_info(link, url_template, apikey, counter)

        # Update company result lists for current company
        names.append(name.lower())
        comp_numbers.append(comp_number)
        addresses.append(address)
        kinds.append(kind)
        creation_dates.append(creation_date)   
        dissolved_dates.append(dissolved_date)
                    
        # Fetch officer information
        num_active, num_inactive, num_resigned, officer_name, officer_role, officer_type, officer_status,\
        officer_country, officer_address, officer_comp_number, counter =\
        fetch_officers(company_num, url_template, apikey, counter)

        # Update officer result lists for current company
        active_officers.append(num_active)
        inactive_officers.append(num_inactive)
        resigned_officers.append(num_resigned)
        officer_names.append(officer_name)
        officer_roles.append(officer_role)
        officer_types.append(officer_type)
        officer_statuses.append(officer_status)
        officer_countries.append(officer_country)
        officer_addresses.append(officer_address)
        officer_comp_numbers.append(officer_comp_number)

        # Fetch PSC information
        psc_active, psc_ceased, psc_type, psc_status, psc_country, psc_comp_number, counter =\
        fetch_psc(company_num, url_template, apikey, counter)

        # Update PSC result lists for current company
        active_pscs.append(psc_active)
        ceased_pscs.append(psc_ceased)
        psc_types.append(psc_type)
        psc_statuses.append(psc_status)
        psc_countries.append(psc_country)
        psc_comp_numbers.append(psc_comp_number)
        
        # Fetch filing history
        filing_comp_number, filing_date, filing_description, filing_name_change, counter =\
        fetch_filings(company_num, url_template, apikey, counter)
        
        filing_comp_numbers.append(filing_comp_number)
        filing_dates.append(filing_date)
        filing_descriptions.append(filing_description)
        filing_name_changes.append(filing_name_change)
                    

    # Flatten lists
    officer_names = [i for sublist in officer_names for i in sublist]
    officer_roles = [i for sublist in officer_roles for i in sublist]
    officer_types = [i for sublist in officer_types for i in sublist]
    officer_statuses = [i for sublist in officer_statuses for i in sublist]
    officer_countries = [i for sublist in officer_countries for i in sublist]
    officer_addresses = [i for sublist in officer_addresses for i in sublist]
    officer_comp_numbers = [i for sublist in officer_comp_numbers for i in sublist]
    psc_types = [i for sublist in psc_types for i in sublist]
    psc_statuses = [i for sublist in psc_statuses for i in sublist]
    psc_countries = [i for sublist in psc_countries for i in sublist]
    psc_comp_numbers = [i for sublist in psc_comp_numbers for i in sublist]
    filing_comp_numbers = [i for sublist in filing_comp_numbers for i in sublist]
    filing_dates = [i for sublist in filing_dates for i in sublist]
    filing_descriptions = [i for sublist in filing_descriptions for i in sublist]


    # Create company results dataframe
    company_results = pd.DataFrame({'name':names,
                    'address':addresses,
                    'kind':kinds,
                    'company_number':comp_numbers,
                    'creation_date':creation_dates,
                    'dissolved_date':dissolved_dates,
                    'active_officers':active_officers,
                    'inactive_officers':inactive_officers,
                    'resigned_officers': resigned_officers,
                    'active_psc': active_pscs,
                    'ceased_psc': ceased_pscs})

    # Create officer results dataframe
    officer_results = pd.DataFrame({'company_number': officer_comp_numbers,
                                    'name': officer_names,
                                    'role': officer_roles,
                                    'type': officer_types,
                                    'status': officer_statuses,
                                    'country': officer_countries,
                                    'address': officer_addresses})
    
    # Create PSC results dataframe
    psc_results = pd.DataFrame({'company_number': psc_comp_numbers,
                            'status': psc_statuses,
                            'type': psc_types,
                            'country': psc_countries})
    
    # Create filing results dataframe
    filing_results = pd.DataFrame({'company_number': filing_comp_numbers,
                                  'filing_date': filing_dates,
                                  'description': filing_descriptions,
                                  'name_change': filing_name_changes})


    return company_results, officer_results, psc_results, filing_results, counter

In [9]:
# Collect all companies linked to an officer

def fetch_company_from_officer(officer_link, url_template, apikey, counter):
    # Instantiate results list
    comp_numbers = []
    
    # Formulate url and make request
    url = url_template.format(officer_link)
    r, counter = request_api(url, apikey, counter)
    
    # Iterate through all companies for officer
    for i in range(len(r['items'])):
        
        # Extract company number from link
        comp_number = r['items'][i]['links']['company'].split('/')[-1]
        # Update results list with current company number
        comp_numbers.append(comp_number)
        
    return comp_numbers, counter

In [10]:
# Perform search for companies from list of officers 

def officer_search(officer_list, url_template, apikey, counter):

    # Define company result lists
    names = []
    comp_numbers = []
    addresses = []
    kinds = []
    creation_dates = []
    dissolved_dates = []
    active_officers = []
    inactive_officers = []
    resigned_officers = []
    active_pscs = []
    ceased_pscs = []

    # Define officer result lists
    officer_names = []
    officer_roles = []
    officer_types = []
    officer_statuses = []
    officer_countries = []
    officer_addresses = []
    officer_comp_numbers = []
    
    # Define psc result lists
    psc_types = []
    psc_statuses = []
    psc_countries = []
    psc_comp_numbers = []
    
    # Define filing history result lists
    filing_dates = []
    filing_descriptions = []
    filing_comp_numbers = []
    filing_name_changes = []

    # Iterate through officers
    for officer in tqdm_notebook(officer_list):
        
        # Initialise list of company numbers collected from officer
        comp_list = []
        
        # Intialise page count
        start_index = 0
        
        # Formulate url and make request
        suffix = '/search/officers?q="{}"&start_index={}'.format(officer.replace(' ', '+'), start_index)
        url = url_template.format(suffix)
        r, counter = request_api(url, apikey, counter)
        
        # Find officer title, remove double spaces
        title = r['items'][0]['title'].strip().replace('  ', ' ') 
        
        # Stop searching when result no longer matches searched name
        while officer.lower() in title.lower():
            
            # Check number of results on page
            r_items = min(r['items_per_page'], r['total_results'])
            
            # Iterate through results
            for i in tqdm_notebook(range(r_items)):
                
                # Find title of current search result
                title = r['items'][i]['title'].strip().replace('  ', ' ')
                    
                # Collect list of company numbers
                comp_nums, counter = fetch_company_from_officer(link, url_template, apikey, counter)
                comp_list.append(comp_nums)
            
            # Check if final page
            if r['page_number']*r['items_per_page'] < r['total_results']:
                
                # Increment to next page of results
                start_index += 20
                
                # Formulate url and make request
                suffix = '/search/officers?q="{}"&start_index={}'.format(officer.replace(' ', '+'), start_index)
                url = url_template.format(suffix)
                r, counter = request_api(url, apikey, counter)
                
                # Find title of first result
                title = r['items'][0]['title'].strip().replace('  ', ' ')
            
            # Stop once final page reached
            else:
                break
        
        # Flatten list
        comp_list = [i for sublist in comp_list for i in sublist]
        
        # Iterate through result list of company numbers
        for comp_num in tqdm_notebook(comp_list):

            # Fetch company information
            link = '/company/'+ comp_num
            name, comp_number, address, kind, creation_date, dissolved_date, counter =\
            fetch_comp_info(link, url_template, apikey, counter)

            # Update company result lists for current company
            names.append(name.lower())
            comp_numbers.append(comp_number)
            addresses.append(address)
            kinds.append(kind)
            creation_dates.append(creation_date)  
            dissolved_dates.append(dissolved_date)

            # Fetch officer information
            num_active, num_inactive, num_resigned, officer_name, officer_role, officer_type, officer_status,\
            officer_country, officer_address, officer_comp_number, counter =\
            fetch_officers(comp_number, url_template, apikey, counter)

            # Update officer result lists for current company
            active_officers.append(num_active)
            inactive_officers.append(num_inactive)
            resigned_officers.append(num_resigned)
            officer_names.append(officer_name)
            officer_roles.append(officer_role)
            officer_types.append(officer_type)
            officer_statuses.append(officer_status)
            officer_countries.append(officer_country)
            officer_addresses.append(officer_address)
            officer_comp_numbers.append(officer_comp_number)

            # Fetch PSC information
            psc_active, psc_ceased, psc_type, psc_status, psc_country, psc_comp_number, counter =\
            fetch_psc(comp_number, url_template, apikey, counter)

            # Update PSC result lists for current company
            active_pscs.append(psc_active)
            ceased_pscs.append(psc_ceased)
            psc_types.append(psc_type)
            psc_statuses.append(psc_status)
            psc_countries.append(psc_country)
            psc_comp_numbers.append(psc_comp_number)
            
            # Fetch filing history
            filing_comp_number, filing_date, filing_description, filing_name_change, counter =\
            fetch_filings(comp_number, url_template, apikey, counter)

            # Update filing result lists for current company
            filing_comp_numbers.append(filing_comp_number)
            filing_dates.append(filing_date)
            filing_descriptions.append(filing_description)
            filing_name_changes.append(filing_name_change)
                    

    # Flatten lists
    officer_names = [i for sublist in officer_names for i in sublist]
    officer_roles = [i for sublist in officer_roles for i in sublist]
    officer_types = [i for sublist in officer_types for i in sublist]
    officer_statuses = [i for sublist in officer_statuses for i in sublist]
    officer_countries = [i for sublist in officer_countries for i in sublist]
    officer_addresses = [i for sublist in officer_addresses for i in sublist]
    officer_comp_numbers = [i for sublist in officer_comp_numbers for i in sublist]
    psc_types = [i for sublist in psc_types for i in sublist]
    psc_statuses = [i for sublist in psc_statuses for i in sublist]
    psc_countries = [i for sublist in psc_countries for i in sublist]
    psc_comp_numbers = [i for sublist in psc_comp_numbers for i in sublist]
    filing_comp_numbers = [i for sublist in filing_comp_numbers for i in sublist]
    filing_dates = [i for sublist in filing_dates for i in sublist]
    filing_descriptions = [i for sublist in filing_descriptions for i in sublist]


    # Create company results dataframe
    company_results = pd.DataFrame({'name':names,
                    'address':addresses,
                    'kind':kinds,
                    'company_number':comp_numbers,
                    'creation_date':creation_dates,
                    'dissolved_date':dissolved_dates,
                    'active_officers':active_officers,
                    'inactive_officers':inactive_officers,
                    'resigned_officers': resigned_officers,
                    'active_psc': active_pscs,
                    'ceased_psc': ceased_pscs})

    # Create officer results dataframe
    officer_results = pd.DataFrame({'company_number': officer_comp_numbers,
                                    'name': officer_names,
                                    'role': officer_roles,
                                    'type': officer_types,
                                    'status': officer_statuses,
                                    'country': officer_countries,
                                    'address': officer_addresses})
    
    # Create PSC results dataframe
    psc_results = pd.DataFrame({'company_number': psc_comp_numbers,
                            'type': psc_types,
                            'status': psc_statuses,
                            'country': psc_countries})
    
    # Create filing results dataframe
    filing_results = pd.DataFrame({'company_number': filing_comp_numbers,
                                   'filing_date': filing_dates,
                                   'description': filing_descriptions,
                                   'name_change':filing_name_changes})


    return company_results, officer_results, psc_results, filing_results, counter

# Read in data from research

In [12]:
# Import list of names for flagged companies
flagged_comp_list = aml_private.flagged_comps

In [81]:
# Import list of names for flagged officers
flagged_list_officers = aml_private.flagged_officers

In [32]:
# Read in laundromat file
ml_df = pd.read_excel('Resource_files/laundromat-companies.xlsx')

# Remove entries with no name
ml_df.dropna(subset=['name'], inplace=True)

# Select only companies linked to at least 10 transactions and where both sending and receiving
ml_df = ml_df[(ml_df.tx_count>10)&(ml_df.amount_usd_in>0)&(ml_df.amount_usd_out>0)]

# Select only company names 
ml_list = ml_df.reset_index(drop=True).name

# Collect positive samples

In [53]:
# Scrape companies from flagged companies list
flagged_comps, flagged_officers, flagged_psc, flagged_filings, request_count =\
company_search(flagged_comp_list, url_template, apikey, request_count)

# Scrape companies from flagged officers list
flagged_comps_2, flagged_officers_2, flagged_psc_2, flagged_filings_2, request_count =\
officer_search(flagged_list_officers, url_template, apikey, request_count)

# Combine flagged results
flagged_comps = pd.concat([flagged_comps, flagged_comps_2]).reset_index(drop=True)
flagged_offs = pd.concat([flagged_offs, flagged_offs_2]).reset_index(drop=True)
flagged_psc = pd.concat([flagged_psc, flagged_psc_2]).reset_index(drop=True)
flagges_filings = pd.concat([flagged_filings, flagged_filings_2]).reset_index(drop=True)

In [109]:
# Scrape fraudulent companies from laundromat file
laund_comps, laund_officers, laund_psc, laund_filings, request_count =\
company_search(ml_list, url_template, apikey, request_count)

{'count': 1, 'reset_time': datetime.datetime(2020, 2, 26, 1, 3, 45, 383903)}



In [116]:
# Read in company factory files
cornwall_df = pd.read_excel('Resource_files/Cornwall_Buildings.xlsx')
cornwall_list = cornwall_df[['Registration number']]
cornwall_list = cornwall_list.reset_index(drop=True)

darkes_df = pd.read_excel('Resource_files/Darkes_Lane.xlsx')
darkes_list = darkes_df[['Registration number']]
darkes_list = darkes_list.reset_index(drop=True)

churchill_df = pd.read_excel('Resource_files/Churchill_Court.xlsx')
churchill_list = churchill_df[['Registration number']]
churchill_list = churchill_list.reset_index(drop=True)

coburg_df = pd.read_excel('Resource_files/Coburg_Road.xlsx')
coburg_list = coburg_df[['Registration number']]
coburg_list = coburg_list.reset_index(drop=True)

# Create list from all company numbers
company_factories = list(pd.concat([cornwall_list, darkes_list, coburg_list])['Registration number'])

# Scrape companies from company factory files
factory_comps, factory_officers, factory_psc, factory_filings, request_count =\
company_number_search(company_factories, url_template, apikey, request_count)

{'count': 1, 'reset_time': datetime.datetime(2020, 2, 26, 9, 7, 12, 221055)}
{'count': 1, 'reset_time': datetime.datetime(2020, 2, 26, 9, 12, 13, 232351)}
{'count': 1, 'reset_time': datetime.datetime(2020, 2, 26, 9, 17, 14, 241090)}
{'count': 1, 'reset_time': datetime.datetime(2020, 2, 26, 9, 22, 15, 204973)}
{'count': 1, 'reset_time': datetime.datetime(2020, 2, 26, 9, 27, 16, 215599)}
{'count': 1, 'reset_time': datetime.datetime(2020, 2, 26, 9, 32, 17, 225899)}
{'count': 1, 'reset_time': datetime.datetime(2020, 2, 26, 9, 37, 18, 210785)}
{'count': 1, 'reset_time': datetime.datetime(2020, 2, 26, 9, 42, 19, 221900)}
{'count': 1, 'reset_time': datetime.datetime(2020, 2, 26, 9, 47, 20, 230173)}
{'count': 1, 'reset_time': datetime.datetime(2020, 2, 26, 9, 52, 21, 217042)}
{'count': 1, 'reset_time': datetime.datetime(2020, 2, 26, 9, 57, 22, 224800)}
{'count': 1, 'reset_time': datetime.datetime(2020, 2, 26, 10, 2, 23, 232288)}



In [142]:
# Combine all positive cases
ml_comps = pd.concat([flagged_comps,laund_comps,factory_comps], sort=False).reset_index(drop=True)

ml_officers = pd.concat([flagged_officers,laund_officers,factory_officers], sort=False).reset_index(drop=True)

ml_psc = pd.concat([flagged_psc,laund_psc,factory_psc], sort=False).reset_index(drop=True)

ml_filings = pd.concat([flagged_filings,laund_filings,factory_filings], sort=False).reset_index(drop=True)

# Remove duplicates
ml_comps.drop_duplicates(inplace=True)
ml_officers.drop_duplicates(inplace=True)
ml_psc.drop_duplicates(inplace=True)
ml_filings.drop_duplicates(inplace=True)

# Label data as positive for suspicious
ml_comps['ml'] = 1

# Save data to csv
ml_comps.to_csv('ml_companies.csv', index=False)
ml_officers.to_csv('ml_officers.csv', index=False)
ml_psc.to_csv('ml_psc.csv', index=False)
ml_filings.to_csv('ml_filings.csv', index=False)

# Count total results
ml_rows = ml_comps.shape[0]

# Collect negative samples 

In [ ]:
# Read in files for entire CH database
non_ml_df_1 = pd.read_csv('Resource_files/BasicCompanyData-2020-02-01-part1_6.csv')
non_ml_df_2 = pd.read_csv('Resource_files/BasicCompanyData-2020-02-01-part2_6.csv')
non_ml_df_3 = pd.read_csv('Resource_files/BasicCompanyData-2020-02-01-part3_6.csv')
non_ml_df_4 = pd.read_csv('Resource_files/BasicCompanyData-2020-02-01-part4_6.csv')
non_ml_df_5 = pd.read_csv('Resource_files/BasicCompanyData-2020-02-01-part5_6.csv')
non_ml_df_6 = pd.read_csv('Resource_files/BasicCompanyData-2020-02-01-part6_6.csv')

# Combine into single dataframe
non_ml_df = pd.concat([non_ml_df_1,\
                       non_ml_df_2,\
                       non_ml_df_3,\
                       non_ml_df_4,\
                       non_ml_df_5,\
                       non_ml_df_6], sort=False).reset_index(drop=True)

# Remove companies without registered address
non_ml_df = non_ml_df[non_ml_df['RegAddress.AddressLine1'].notnull()]

In [165]:
# Randomly sample n+10 company numbers as redundancy in case of overlap with ml companies
sample_comp_nums = list(non_ml_df.sample(n=(ml_rows+10)).loc[:, ' CompanyNumber'])

# Check if any of random sample appear in positive cases and remove if so 
overlaps = ml_comps[ml_comps['company_number'].isin(sample_comp_nums)].company_number

# Remove by company number if overlap in positive cases
if len(overlaps) > 0:
    for overlap in overlaps:
        sample_comp_nums.remove(overlap)
    
# Remove excess companies to match positive sample size
excess = 10-len(overlaps)
    for i in range(excess):
        sample_comp_nums.pop()

In [ ]:
# Scrape non-fraudulent companies
non_ml_comps, non_ml_officers, non_ml_psc, non_ml_filings, request_count =\
company_number_search(sample_comp_nums, url_template, apikey, request_count)

# Label data as negative, ie not suspicious
non_ml_comps['ml'] = 0

# Save data to csv
non_ml_comps.to_csv('non_ml_companies.csv', index=False)
non_ml_officers.to_csv('non_ml_officers.csv', index=False)
non_ml_psc.to_csv('non_ml_psc.csv', index=False)
non_ml_filings.to_csv('non_ml_filings.csv', index=False)

In [ ]:
# Save data to csv
non_ml_comps.to_csv('non_ml_companies.csv', index=False)
non_ml_officers.to_csv('non_ml_officers.csv', index=False)
non_ml_psc.to_csv('non_ml_psc.csv', index=False)
non_ml_filings.to_csv('non_ml_filings.csv', index=False)